In [148]:
import sqlalchemy
from sqlalchemy import *
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy.orm.session import Session
from sqlalchemy import func
from sqlalchemy.sql import operators, extract
from sqlalchemy import asc, desc, inspect
from sqlalchemy.orm import aliased, contains_eager
from sqlalchemy.ext.declarative import declarative_base
import pandas as pd

In [149]:
db = create_engine('sqlite:///chinook.db')
meta = MetaData(db)
meta.reflect(bind=db)

invoice_items = meta.tables['invoice_items']
invoices = meta.tables['invoices']
customers = meta.tables['customers']
tracks = meta.tables['tracks']
albums = meta.tables['albums']
artists = meta.tables['artists']

In [150]:
from sqlalchemy.orm import scoped_session, sessionmaker
Session = sessionmaker(bind = db)
session = Session()

subquery = session.query(
    extract('year', invoices.c.InvoiceDate).label('Year'), func.sum(invoice_items.c.UnitPrice*invoice_items.c.Quantity).label('revenue'), artists.c.Name, artists.c.ArtistId,
    func.rank().over(
        partition_by = extract('year', invoices.c.InvoiceDate),
        order_by = func.sum(invoice_items.c.UnitPrice*invoice_items.c.Quantity).desc()).label('rank')).join(
    invoice_items, invoice_items.c.InvoiceId == invoices.c.InvoiceId).join(
    tracks, tracks.c.TrackId == invoice_items.c.TrackId).join(
    albums, albums.c.AlbumId == tracks.c.AlbumId).join(
    artists, artists.c.ArtistId == albums.c.ArtistId).filter(
    invoices.c.BillingState == 'CA').group_by(
    extract('year', invoices.c.InvoiceDate), artists.c.ArtistId).subquery()

query = session.query(subquery).filter(subquery.c.rank <= 3)

df = pd.DataFrame(query)
df

/Users/jeongjaehui/opt/anaconda3/envs/env-stats404-w20/lib/python3.9/site-packages/sqlalchemy/sql/sqltypes.py:661: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  util.warn(


,Year,revenue,Name,ArtistId,rank
0,2009,4.9500000000,The Cult,139,1
1,2009,3.9600000000,Deep Purple,58,2
2,2009,2.9700000000,The Clash,138,3
3,2009,2.9700000000,Eric Clapton,81,3
4,2009,2.9700000000,Djavan,80,3
5,2010,13.8600000000,Iron Maiden,90,1
6,2010,1.9800000000,Eric Clapton,81,2
7,2010,1.9800000000,Toquinho & Vinícius,70,2
8,2010,1.9800000000,Gene Krupa,69,2
9,2010,1.9800000000,Miles Davis,68,2
